### This jupyter notebook employs a fully connective neural network(FC) or its alias artificial neural network (ANN) to learn the mapping between input current configuration between output magnetic field 

In [46]:
%reload_ext autoreload
%autoreload 2
import numpy
import torch
if torch.cuda.device_count():
    device = 'cuda'
    print('Good to go')
else:
    device = 'cpu'
    print('Using cpu')

Using cpu


/home/qubot/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [43]:
from ReadData import ReadFolder 
import glob
import os
# print(os.getcwd())
foldername="./Data/"
filepattern = "MagneticField[0-9]*.txt"
data = ReadFolder(foldername,filepattern)
fileList = glob.glob(foldername+filepattern)
position = data[:,:3]
Bfield = data[:,3:]
print(fileList)
print(data.shape)


['./Data/MagneticField0.txt']
torch.Size([9260, 6, 1])


In [45]:
from Neural_network import NN_net, Plain_fc_block, weight_init, eMNS_Dataset
from Training_loop import train_part


batch_size = 128
# construct dataset
dataset = eMNS_Dataset(
    train_x=position,
    train_y=Bfield
)

# split the dataset to train, validation, test
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [0.7,0.2,0.1])

#Using Dataloader for batch train
train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_set,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=True)


###############################################
# Config the neural network
###############################################
num_input = 15
num_output = 3
fc_stages = [(num_input,100,1),(100,50,1),(50,25,1)]
fc_network = NN_net(None,fc_stages,None,Plain_fc_block, num_output=num_output)
epochs = 50
learning_rate_decay = .1
learning_rates = [1e-3]
schedule = []
weight_decays = [0]


################################################
# Train the neural network
################################################

for learning_rate in learning_rates:
    for weight_decay in weight_decays:
        fc_network.apply(weight_init)
        optimizer = torch.optim.Adam([{'params':fc_network.parameters()}], lr=learning_rate, weight_decay= weight_decay)
        RMSE_history, RMSE_val_history, loss_history, iter_history = train_part(model=fc_network, optimizer=optimizer, train_loader=train_loader, valid_loader=valid_loader, epochs=epochs, learning_rate_decay=learning_rate_decay, schedule=schedule, weight_decay=weight_decay, verbose=False, device=device)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x3 and 11x100)